<center><img src="images/DLI_Header.png" alt="标题" style="width: 400px;"/></center>

# 2. 扩散模型

在之前的笔记本中，我们学习了如何使用 U-Net 从图像中分离噪声，但它无法从噪声中生成可信的新图像。扩散模型在从头开始生成图像方面要好得多。

好消息是，我们的神经网络模型不会有太大变化。我们将在 U-Net 架构的基础上进行一些细微的修改。

相反，最大的区别在于我们如何使用我们的模型。我们不会一次性将噪声添加到图像中，而是多次添加少量噪声。然后，我们可以多次在噪声图像上使用我们的神经网络来生成新图像，如下所示：

<center><img src="images/rev_diffusion.png" /></center>

#### 学习目标

此笔记本的目标是：
* 构建正向扩散方差计划
* 定义正向扩散函数 `q`
* 更新 U-Net 架构以适应时间步长 `t`
* 训练模型以根据时间步长 `t` 检测添加到图像中的噪声
* 定义反向扩散函数以模拟 `p`
* 尝试生成服装（再次）

我们已将上一个笔记本中的一些函数移至 [other_utils.py](utils/other_utils.py) 文件中。我们可以使用它来重新加载 fashionMNIST 数据集：

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim import Adam

# Visualization tools
import matplotlib.pyplot as plt
from IPython.display import Image

# User defined libraries
from utils import other_utils

IMG_SIZE = 16
IMG_CH = 1
BATCH_SIZE = 128
data, dataloader = other_utils.load_transformed_fashionMNIST(IMG_SIZE, BATCH_SIZE)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## 2.1 正向扩散

让 `T` 成为我们将向图像添加噪声的次数。我们可以使用 `t` 来跟踪当前的 `timestep`。

在之前的笔记本中，我们使用术语 `beta` 来表示新图像与原始图像相比的噪声百分比。默认值为 50% 噪声和 50% 原始图像。这次，我们将使用 `variance schedule`，表示为 $\beta_t$，或代码中的 `B`。这将描述在每个时间步 `t` 中将向我们的图像添加多少噪声。

在论文 [Denoising Diffusion Probabilistic Models](https://arxiv.org/abs/2006.11239?ref=assemblyai.com) 的第 4 部分中，作者讨论了定义良好时间表的技巧。它应该足够大，以便模型能够识别已添加的噪声（特别是因为图像可能已经很嘈杂），但仍然尽可能小。

In [ ]:
nrows = 10
ncols = 15

T = nrows * ncols
start = 0.0001
end = 0.02
B = torch.linspace(start, end, T).to(device)
B

[正态分布](https://mathworld.wolfram.com/NormalDistribution.html) 具有以下签名：

$\mathcal{N}(x;u,\sigma^2)$ = $\frac{1}{\sigma\sqrt{2\pi}}\mathcal{e}^{-\frac{1}{2}\left(\frac{x-\mu}{\sigma}\right)^{2}}$

其含义为“参数为 $u$（均值）和 $\sigma^2$（方差）的 $x$ 正态分布”。当 $\mu$ 为 0 且 $\sigma$ 为 1 时，我们得到标准正态分布 $\mathcal{N}(x;0,1)$，其概率密度如下图所示：

<center><img src="images/normal.png" /></center>

如果我们改变图像由于噪声在多个时间步骤中多次出现，我们将 $\mathbf{x}_{t}$ 描述为时间步骤 $t$ 处的图像。然后，$\mathbf{x}_{t-1}$ 将是前一个时间步骤处的图像，而 $x_{0}$ 将是原始图像。

在之前的笔记本中，我们使用以下公式向图像添加噪声：

$q(\mathbf{x}_{t}|\mathbf{x}_{t-1})=\mathcal{N}(\mathbf{x}_{t};(1-\beta_{t}) \cdot \mathbf{x}_{t-1},\beta_{t}^{2} \cdot \mathbf{I})$

其中 $q$ 表示 [正向扩散过程](https://lilianweng.github.io/posts/2021-07-11-diffusion-models/#forward-diffusion-process) ，$q(\mathbf{x}_{t}|\mathbf{x}_{t-1})$ 描述了基于 $\mathbf{x}_{t-1}$ 的新噪声图像 $\mathbf{x}_{t}$ 的概率分布。

这次，我们将使用类似的方程式来改变图像：

$q(\mathbf{x}_{t}|\mathbf{x}_{t-1})=\mathcal{N}(\mathbf{x}_{t};\sqrt{1-\beta_{t}} \cdot \mathbf{x}_{t-1},\beta_{t} \cdot \mathbf{I})$

我们可以从这个概率分布中抽样，首先使用 [torch.randn_like](https://pytorch.org/docs/stable/generated/torch.randn_like.html) 从标准正态分布 $\mathcal{N}(x;0,1)$ 中抽样：

`noise = torch.randn_like(x_t)`

然后我们可以将噪声相乘并添加到 `q` 中以进行抽样：

`x_t = torch.sqrt(1 - B[t]) * x_t + torch.sqrt(B[t]) * noise`

让我们在实践中看看这一切。运行下面的代码单元，对我们数据集的第一个图像执行 `T` （或 `150` ）次正向扩散。

In [ ]:
plt.figure(figsize=(8, 8))
x_0 = data[0][0].to(device)  # Initial image
x_t = x_0  # Set up recursion
xs = []  # Store x_t for each T to see change

for t in range(T):
    noise = torch.randn_like(x_t)
    x_t = torch.sqrt(1 - B[t]) * x_t + torch.sqrt(B[t]) * noise  # sample from q(x_t|x_t-1)
    img = torch.squeeze(x_t).cpu()
    xs.append(img)
    ax = plt.subplot(nrows, ncols, t + 1)
    ax.axis("off")
    plt.imshow(img)
plt.savefig("forward_diffusion.png", bbox_inches="tight")

或者以动画形式：

In [ ]:
gif_name = "forward_diffusion.gif"
other_utils.save_animation(xs, gif_name)

In [ ]:
Image(open(gif_name,'rb').read())

## 2.2 省略噪声处理

我们可以对数据集中的每个图像进行 `T` 次噪声处理，以创建 `T` 个新图像，但我们需要这样做吗？

借助递归的力量，我们可以估算出给定我们的测试计划 $\beta_t$ 后 $x_t$ 会是什么样子。完整的数学分解可以在 [Lilian Weng 的博客](https://lilianweng.github.io/posts/2021-07-11-diffusion-models/#speed-up-diffusion-model-sampling) 中找到。
让我们带回 `a`lpha，它是 $\beta$ 的补充。我们可以将 $\alpha_t$ 定义为 $1 - \beta_t$，我们可以将 $\bar{\alpha}_t$ 定义为 $\alpha_t$ 的 [累积乘积](https://pytorch.org/docs/stable/generated/torch.cumprod.html)。

例如，$\bar{\alpha}_3 = \alpha_0 \cdot \alpha_1 \cdot \alpha_2 \cdot \alpha_3$

由于符号为条形，我们将 $\bar{\alpha}_t$ 称为 `a_bar`。我们新的噪声图像分布变为：

$q(\mathbf{x}_{t}|\mathbf{x}_{0})=\mathcal{N}(\mathbf{x}_{t};\sqrt{\bar{\alpha}_{t}} \cdot x_{0},(1 - \bar{\alpha}_t) \cdot \mathbf{I})$

转换为代码如下：

`x_t = sqrt_a_bar_t * x_0 + sqrt_one_minus_a_bar_t * noise`

我们现在不再依赖 $\mathbf{x}_{t-1}$，可以从 $x_0$ 估计 $\mathbf{x}_t$。让我们在代码中定义这些变量：

In [ ]:
a = 1. - B
a_bar = torch.cumprod(a, dim=0)
sqrt_a_bar = torch.sqrt(a_bar)  # Mean Coefficient
sqrt_one_minus_a_bar = torch.sqrt(1 - a_bar) # St. Dev. Coefficient

我们已经准备好了所有要素，让我们来编写前向扩散采样函数 `q`：

$q(\mathbf{x}_{t}|\mathbf{x}_{0})=\mathcal{N}(\mathbf{x}_{t};\sqrt{\bar{\alpha}_{t}} \cdot \mathbf{x}_{0},(1 - \bar{\alpha}_t) \cdot \mathbf{I})$

目前，`sqrt_a_bar` 和 `sqrt_one_minus_a_bar` 只有一个维度，如果我们用 `t` 索引它们，它们每个都只有一个值。如果我们想将此值与图像中的每个像素值相乘，我们需要匹配维度数才能[广播](https://numpy.org/doc/stable/user/basics.broadcasting.html)。

<center><img src="images/broadcasting.png" width="60%" /></center>

我们可以通过使用 `None` 进行索引来添加额外的维度。这是 PyTorch 的快捷方式，用于向结果张量添加额外的维度。作为参考，一批图像的尺寸为： `批次维度 x 图像通道 x 图像高度 x 图像宽度` 。

In [ ]:
def q(x_0, t):
    """
    Samples a new image from q
    Returns the noise applied to an image at timestep t
    x_0: the original image
    t: timestep
    """
    t = t.int()
    noise = torch.randn_like(x_0)
    sqrt_a_bar_t = sqrt_a_bar[t, None, None, None]
    sqrt_one_minus_a_bar_t = sqrt_one_minus_a_bar[t, None, None, None]

    x_t = sqrt_a_bar_t * x_0 + sqrt_one_minus_a_bar_t * noise
    return x_t, noise

让我们测试一下这种新方法，并将其与旧的递归生成图像的方法进行比较。

In [ ]:
plt.figure(figsize=(8, 8))
xs = []

for t in range(T):
    t_tenser = torch.Tensor([t]).type(torch.int64)
    x_t, _ = q(x_0, t_tenser)
    img = torch.squeeze(x_t).cpu()
    xs.append(img)
    ax = plt.subplot(nrows, ncols, t + 1)
    ax.axis('off')
    other_utils.show_tensor_image(x_t)
plt.savefig("forward_diffusion_skip.png", bbox_inches='tight')

In [ ]:
gif_name = "forward_diffusion_skip.gif"
other_utils.save_animation(xs, gif_name)

In [ ]:
Image(open(gif_name,'rb').read())

与之前的技术相比，你看出来有什么不同吗？当按顺序添加噪声时，连续时间步长的图像之间的差异较小。尽管如此，神经网络在反向扩散过程中仍能很好地将噪声与原始图像分离。

## 2.3 预测噪声

我们的神经网络架构与之前基本相同。但是，由于添加的噪声量会随着每个时间步骤而变化，因此我们需要一种方法来告诉模型我们的输入图像处于哪个时间步骤。

为此，我们可以创建一个如下所示的嵌入块。
* `input_dim` 是我们想要嵌入的值的维数。我们将嵌入 `t`，它是一个一维标量。
* `emb_dim` 是我们希望使用 [Linear](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html) 层将输入值转换为的维数。
* [UnFlatten](https://pytorch.org/docs/stable/generated/torch.nn.Unflatten.html) 用于将向量重塑为多维空间。由于我们要将此嵌入的结果添加到多维特征图中，因此我们将添加一些额外的维度，类似于我们在上面的 `q` 函数中扩展维度的方式。

In [ ]:
class EmbedBlock(nn.Module):
    def __init__(self, input_dim, emb_dim):
        super().__init__()
        self.input_dim = input_dim
        layers = [
            nn.Linear(input_dim, emb_dim),
            nn.ReLU(),
            nn.Linear(emb_dim, emb_dim),
            nn.Unflatten(1, (emb_dim, 1, 1))
        ]
        self.model = nn.Sequential(*layers)

    def forward(self, input):
        input = input.view(-1, self.input_dim)
        return self.model(input)

我们将把这个时间嵌入块添加到 U-Net 的每个 `UpBlock` 中，从而得到以下架构。

<center><img src="images/time_nn.png" width="80%" /></center>

**TODO**：我们的 `DownBlock` 与之前相同。使用上图作为参考，您能用正确的变量替换 `FIXME` 吗？每个 `FIXME` 可以是以下之一：
* `in_chs`
* `out_chs`
* `kernel_size`
* `stride`
* `padding`

单击下面的 `...` 获取正确答案。

In [ ]:
class DownBlock(nn.Module):
    def __init__(self, in_chs, out_chs):
        kernel_size = 3
        stride = 1
        padding = 1

        super().__init__()
        layers = [
            nn.Conv2d(FIXME, FIXME, FIXME, FIXME, FIXME),
            nn.BatchNorm2d(FIXME),
            nn.ReLU(),
            nn.Conv2d(FIXME, FIXME, FIXME, FIXME, FIXME),
            nn.BatchNorm2d(FIXME),
            nn.ReLU(),
            nn.MaxPool2d(2)
        ]
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

In [ ]:
class DownBlock(nn.Module):
    def __init__(self, in_chs, out_chs):
        kernel_size = 3
        stride = 1
        padding = 1

        super().__init__()
        layers = [
            nn.Conv2d(in_chs, out_chs, kernel_size, stride, padding),
            nn.BatchNorm2d(out_chs),
            nn.ReLU(),
            nn.Conv2d(out_chs, out_chs, kernel_size, stride, padding),
            nn.BatchNorm2d(out_chs),
            nn.ReLU(),
            nn.MaxPool2d(2)
        ]
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

`UpBlock` 遵循类似的逻辑，但使用的是 [转置卷积](https://pytorch.org/docs/stable/generated/torch.nn.ConvTranspose2d.html)。

**TODO**：您能用正确的变量替换 `FIXME` 吗？每个 `FIXME` 可以是以下之一：
* `in_chs`
* `out_chs`
* `kernel_size`
* `stride`
* `padding`
* `strideT`
* `out_paddingT`
* `x`
* `skip`

单击下面的 `...` 获取正确答案。

In [ ]:
class UpBlock(nn.Module):
    def __init__(self, in_chs, out_chs):
        # Convolution variables
        kernel_size = 3
        stride = 1
        padding = 1

        # Transpose variables
        strideT = 2
        out_paddingT = 1

        super().__init__()
        # 2 * in_chs for concatenated skip connection
        layers = [
            nn.ConvTranspose2d(FIXME, FIXME, FIXME, FIXME, FIXME, FIXME),
            nn.BatchNorm2d(FIXME),
            nn.ReLU(),
            nn.Conv2d(FIXME, FIXME, FIXME, FIXME, FIXME),
            nn.BatchNorm2d(FIXME),
            nn.ReLU()
        ]
        self.model = nn.Sequential(*layers)
    
    def forward(self, x, skip):
        x = torch.cat((FIXME, FIXME), 1)
        x = self.model(FIXME)
        return x

In [ ]:
class UpBlock(nn.Module):
    def __init__(self, in_chs, out_chs):
        # Convolution variables
        kernel_size = 3
        stride = 1
        padding = 1

        # Transpose variables
        strideT = 2
        out_paddingT = 1

        super().__init__()
        # 2 * in_chs for concatenated skip connection
        layers = [
            nn.ConvTranspose2d(2 * in_chs, out_chs, kernel_size, strideT, padding, out_paddingT),
            nn.BatchNorm2d(out_chs),
            nn.ReLU(),
            nn.Conv2d(out_chs, out_chs, kernel_size, stride, padding),
            nn.BatchNorm2d(out_chs),
            nn.ReLU()
        ]
        self.model = nn.Sequential(*layers)
    
    def forward(self, x, skip):
        x = torch.cat((x, skip), 1)
        x = self.model(x)
        return x

最终的 U-Net 与我们在上一个笔记本中使用的类似。不同之处在于我们现在有一个连接到 `UpBlock` 的时间嵌入。

**TODO**：虽然时间嵌入已集成到模型中，但仍有许多 `FIXME` 需要替换。这次，图像通道、上行通道和下行通道需要修复。您能否上下移动 U-Net 以在每个步骤中设置正确的通道数？

每个 `FIXME` 可以是：
* `img_chs`
* `down_chs` 中的值
* `up_chs` 中的值

单击下面的 `...` 获取正确答案。

In [ ]:
class UNet(nn.Module):
    def __init__(self):
        super().__init__()
        img_chs = IMG_CH
        down_chs = (16, 32, 64)
        up_chs = down_chs[::-1]  # Reverse of the down channels
        latent_image_size = IMG_SIZE // 4 # 2 ** (len(down_chs) - 1)
        t_dim = 1 # New

        # Inital convolution
        self.down0 = nn.Sequential(
            nn.Conv2d(FIXME, down_chs[0], 3, padding=1),
            nn.BatchNorm2d(FIXME),
            nn.ReLU()
        )

        # Downsample
        self.down1 = DownBlock(down_chs[0], down_chs[1])
        self.down2 = DownBlock(FIXME, FIXME)
        self.to_vec = nn.Sequential(nn.Flatten(), nn.ReLU())
        
        # Embeddings
        self.dense_emb = nn.Sequential(
            nn.Linear(FIXME*latent_image_size**2, down_chs[1]),
            nn.ReLU(),
            nn.Linear(down_chs[1], FIXME),
            nn.ReLU(),
            nn.Linear(down_chs[1], down_chs[2]*latent_image_size**2),
            nn.ReLU()
        )
        self.temb_1 = EmbedBlock(t_dim, up_chs[0]) # New
        self.temb_2 = EmbedBlock(t_dim, up_chs[1]) # New
        
        # Upsample
        self.up0 = nn.Sequential(
            nn.Unflatten(1, (FIXME, latent_image_size, latent_image_size)),
            nn.Conv2d(FIXME, up_chs[0], 3, padding=1),
            nn.BatchNorm2d(up_chs[0]),
            nn.ReLU(),
        )
        self.up1 = UpBlock(up_chs[0], up_chs[1])
        self.up2 = UpBlock(FIXME, FIXME)

        # Match output channels
        self.out = nn.Sequential(
            nn.Conv2d(FIXME, FIXME, 3, 1, 1),
            nn.BatchNorm2d(up_chs[-1]),
            nn.ReLU(),
            nn.Conv2d(up_chs[-1], img_chs, 3, 1, 1)
        )

    def forward(self, x, t):
        down0 = self.down0(x)
        down1 = self.down1(down0)
        down2 = self.down2(down1)
        latent_vec = self.to_vec(down2)
        
        latent_vec = self.dense_emb(latent_vec)
        # New
        t = t.float() / T  # Convert from [0, T] to [0, 1]
        temb_1 = self.temb_1(t)
        temb_2 = self.temb_2(t)

        up0 = self.up0(latent_vec)
        up1 = self.up1(up0+temb_1, down2)
        up2 = self.up2(up1+temb_2, down1)
        return self.out(up2)

In [ ]:
class UNet(nn.Module):
    def __init__(self):
        super().__init__()
        img_chs = IMG_CH
        down_chs = (16, 32, 64)
        up_chs = down_chs[::-1]  # Reverse of the down channels
        latent_image_size = IMG_SIZE // 4 # 2 ** (len(down_chs) - 1)
        t_dim = 1 # New

        # Inital convolution
        self.down0 = nn.Sequential(
            nn.Conv2d(img_chs, down_chs[0], 3, padding=1),
            nn.BatchNorm2d(down_chs[0]),
            nn.ReLU()
        )

        # Downsample
        self.down1 = DownBlock(down_chs[0], down_chs[1])
        self.down2 = DownBlock(down_chs[1], down_chs[2])
        self.to_vec = nn.Sequential(nn.Flatten(), nn.ReLU())
        
        # Embeddings
        self.dense_emb = nn.Sequential(
            nn.Linear(down_chs[2]*latent_image_size**2, down_chs[1]),
            nn.ReLU(),
            nn.Linear(down_chs[1], down_chs[1]),
            nn.ReLU(),
            nn.Linear(down_chs[1], down_chs[2]*latent_image_size**2),
            nn.ReLU()
        )
        self.temb_1 = EmbedBlock(t_dim, up_chs[0])  # New
        self.temb_2 = EmbedBlock(t_dim, up_chs[1])  # New
        
        # Upsample
        self.up0 = nn.Sequential(
            nn.Unflatten(1, (up_chs[0], latent_image_size, latent_image_size)),
            nn.Conv2d(up_chs[0], up_chs[0], 3, padding=1),
            nn.BatchNorm2d(up_chs[0]),
            nn.ReLU(),
        )
        self.up1 = UpBlock(up_chs[0], up_chs[1])
        self.up2 = UpBlock(up_chs[1], up_chs[2])

        # Match output channels
        self.out = nn.Sequential(
            nn.Conv2d(up_chs[-1], up_chs[-1], 3, 1, 1),
            nn.BatchNorm2d(up_chs[-1]),
            nn.ReLU(),
            nn.Conv2d(up_chs[-1], img_chs, 3, 1, 1)
        )

    def forward(self, x, t):
        down0 = self.down0(x)
        down1 = self.down1(down0)
        down2 = self.down2(down1)
        latent_vec = self.to_vec(down2)
        
        # New
        t = t.float() / T  # Convert from [0, T] to [0, 1]
        latent_vec = self.dense_emb(latent_vec)
        temb_1 = self.temb_1(t)
        temb_2 = self.temb_2(t)

        up0 = self.up0(latent_vec)
        up1 = self.up1(up0+temb_1, down2)
        up2 = self.up2(up1+temb_2, down1)
        return self.out(up2)

In [ ]:
model = UNet()
print("Num params: ", sum(p.numel() for p in model.parameters()))
model = torch.compile(UNet().to(device))

### 2.3.1 损失函数

在上一个笔记本中，我们使用了 [均方误差](https://developers.google.com/machine-learning/glossary#mean-squared-error-mse) 损失函数，比较了原始图像和基于噪声的预测原始图像。

这次，我们将比较添加到图像中的真实噪声和预测噪声。Lilian Weng 在这篇 [博客文章](https://lilianweng.github.io/posts/2021-07-11-diffusion-models/#parameterization-of-l_t-for-training-loss) 中介绍了数学知识。最初，损失函数基于 [证据下限 (ELBO)](https://en.wikipedia.org/wiki/Evidence_lower_bound) [对数似然](https://mathworld.wolfram.com/Log-LikelihoodFunction.html)，但在 [去噪扩散概率模型论文](https://arxiv.org/abs/2006.11239) 中发现，预测噪声与真实噪声之间的均方误差在实践中更优。如果感兴趣，Lilian Weng 会在 [这里](https://lilianweng.github.io/posts/2021-07-11-diffusion-models/#reverse-diffusion-process) 介绍推导过程。

In [ ]:
def get_loss(model, x_0, t):
    x_noisy, noise = q(x_0, t)
    noise_pred = model(x_noisy, t/T) # Normalize t to be from 0 to 1
    return F.mse_loss(noise, noise_pred)

## 2.4 逆向扩散

我们现在有一个模型可以预测在时间步长 `t` 添加到图像中的噪声，但生成图像并不像反复减去和添加噪声那么简单。`q` 函数可以反转，这样我们就可以从$\mathbf{x}_t$生成$\mathbf{x}_(t-1)$。

$q(\mathbf{x}_{t}|\mathbf{x}_{t-1}) = \mathcal{N}(\mathbf{x}_{t-1};{\mathbf{\tilde{\mu}}}(\mathbf{x_t},\mathbf{x_0}), \tilde{\beta}_t \cdot \mathbf{I})$

**注意**：$\tilde{\beta}_t$最初计算为$\frac{1-\overline{a}_{t-1}}{1-\overline{a}_{t}}\beta_t$，但实际中只使用$\beta_t$更为有效。

使用 [贝叶斯定理](https://en.wikipedia.org/wiki/Bayes%27_theorem)，我们可以得出时间步长 `t` 的模型平均值 `u_t` 的方程。

${\mathbf{\tilde{\mu}}}_t = \frac{1}{\sqrt{\alpha_t}}(\mathbf{x_t}-\frac{1-\alpha_t}{\sqrt{1-\overline{\alpha_t}}}\mathbf{\epsilon}_t)$

图像 $\mathbf{x}_{t-1}$ 可以通过 ${\mathbf{\tilde{\mu}}}_t + \tilde{\beta}_t \cdot \mathbf{I}$ 来估计，因此我们将使用此方程递归生成样本图像，直到达到 `t == 0`。让我们看看这在代码中意味着什么。首先，我们将预先计算计算 `u_t` 所需的值。

In [ ]:
sqrt_a_inv = torch.sqrt(1 / a)
pred_noise_coeff = (1 - a) / torch.sqrt(1 - a_bar)

接下来，我们将创建反向扩散函数 `reverse_q` 。

In [ ]:
@torch.no_grad()
def reverse_q(x_t, t, e_t):
    t = torch.squeeze(t[0].int())  # All t values should be the same
    pred_noise_coeff_t = pred_noise_coeff[t]
    sqrt_a_inv_t = sqrt_a_inv[t]
    u_t = sqrt_a_inv_t * (x_t - pred_noise_coeff_t * e_t)
    if t == 0:
        return u_t  # Reverse diffusion complete!
    else:
        B_t = B[t-1]
        new_noise = torch.randn_like(x_t)
        return u_t + torch.sqrt(B_t) * new_noise

让我们创建一个函数来迭代地从图像中去除噪声，直到图像中没有噪声为止。我们还将显示这些图像，以便我们可以看到模型的改进情况。

In [ ]:
@torch.no_grad()
def sample_images(ncols, figsize=(8,8)):
    plt.figure(figsize=figsize)
    plt.axis("off")
    hidden_rows = T / ncols

    # Noise to generate images from
    x_t = torch.randn((1, IMG_CH, IMG_SIZE, IMG_SIZE), device=device)

    # Go from T to 0 removing and adding noise until t = 0
    plot_number = 1
    for i in range(0, T)[::-1]:
        t = torch.full((1,), i, device=device)
        e_t = model(x_t, t)  # Predicted noise
        x_t = reverse_q(x_t, t, e_t)
        if i % hidden_rows == 0:
            ax = plt.subplot(1, ncols+1, plot_number)
            ax.axis('off')
            other_utils.show_tensor_image(x_t.detach().cpu())
            plot_number += 1
    plt.show()

是时候训练模型了！怎么样？看起来模型正在学习吗？

In [ ]:
optimizer = Adam(model.parameters(), lr=0.001)
epochs = 3
ncols = 15  # Should evenly divide T

model.train()
for epoch in range(epochs):
    for step, batch in enumerate(dataloader):
        optimizer.zero_grad()

        t = torch.randint(0, T, (BATCH_SIZE,), device=device)
        x = batch[0].to(device)
        loss = get_loss(model, x, t)
        loss.backward()
        optimizer.step()

        if epoch % 1 == 0 and step % 100 == 0:
            print(f"Epoch {epoch} | Step {step:03d} | Loss: {loss.item()} ")
            sample_images(ncols)
print("Final sample:")
sample_images(ncols)

如果你眯起眼睛，你能看出模型正在生成什么吗？

In [ ]:
model.eval()
figsize=(8,8) # Change me
ncols = 3 # Should evenly divide T
for _ in range(10):
    sample_images(ncols, figsize=figsize)

## 2.5 下一步

模型正在学习……一些东西。它看起来有点像素化。为什么会这样？继续阅读下一篇笔记本以了解更多信息。

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

<center><img src="images/DLI_Header.png" alt="标题" style="width: 400px;"/></center>